In [1]:
#imports
import math
import random
import pandas as pd
import numpy as np
import os
import networkx as nx

In [2]:
excel_list1 = os.listdir("Raw_Data")
excel_list = []
for i in excel_list1:
    if(i.endswith(".xlsx")):
        excel_list.append(i)      

In [11]:
graphs = []
for i in excel_list:
    path = "Raw_Data/"+i
    excel = pd.read_excel(path,  nrows=1000 ,engine="openpyxl" ,header=0,index_col=0, usecols="A:E", names=["SNo","city1","city2","to","from"] )
#     G = nx.Graph()
    print(path,excel.shape)
#     print(excel["city1"][305])
#     print(asd)
#     for i in range(1,len(excel)-1):
#         G.add_edge(excel["city1"][i],excel["city2"][i],weight=excel["to"][i])
#         G.add_edge(excel["city2"][i],excel["city1"][i],weight=excel["from"][i])
#     print(nx.adjacency_matrix(G).todense())
#     graphs.append(nx.adjacency_matrix(G).todense())

Raw_Data/1115.xlsx (1000, 4)
Raw_Data/0119.xlsx (1000, 4)
Raw_Data/0915.xlsx (1000, 4)
Raw_Data/1119.xlsx (1000, 4)
Raw_Data/1219.xlsx (1000, 4)
Raw_Data/0517.xlsx (1000, 4)


KeyboardInterrupt: 